# Import segmented iceberg maps to btrack and attempt tracking
Perform Bayesian tracking of icebergs using 'btrack'. Much of the code here is copied from the btrack documentation
https://github.com/quantumjot/BayesianTracker

btrack v>=0.5 must be installed from branch 'visual-features'using Python 3.7. Conflicts arise with fiona when using Python 3.8

Iceberg segmentations stored in 'data' folder are sample outputs of a fully unsupervised classifier for Sentinel-1 SAR imagery over a one-year period in 2019-2020.

Initial testing uses a subset defined by the 'Thwaites_Tracking_subset' shapefile in the 'data' folder

In [1]:
import glob
import os
import fiona
import rasterio
import btrack
import napari

import numpy as np
from pathlib import Path
from skimage.io import imread
from skimage.measure import regionprops, regionprops_table, label
from skimage.morphology import remove_small_objects
import DP_Utils


Function to ensure all imported subsets are the same size as the first one processed even if underlying image extent and precise pixel geolocation differs

In [2]:
def pad_or_clip(arr, rows,cols):
    
    if arr.shape[0]>rows:
        arr=arr[0:rows, :]
    elif arr.shape[0]<rows:
        pad_rows=rows-arr.shape[0]
        # npad is a tuple of (n_before, n_after) for each dimension
        npad = ((0, pad_rows), (0, 0))
        arr = np.pad(arr, pad_width=npad, mode='constant', constant_values=0)
        
    if arr.shape[1]>cols:
        arr=arr[:, 0:cols]
    elif arr.shape[1]<cols:
        pad_cols=cols-arr.shape[1]
        npad = ((0,0), (0, pad_cols))
        arr = np.pad(arr, pad_width=npad, mode='constant', constant_values=0)
        
    return arr

Function to construct a numpy array stack from segmentations in specified directory

In [3]:
def segmentation_arr(files, minsize=1):
    """Segmentation as numpy array."""
    
    stack = []
    for filename in files:
        imgWR=DP_Utils.load_to_WorkingRaster(filename, mask=shapes)
        img=imgWR.data
        
        if filename == files[0]: #if first image get the shape to apply to subsequent images
            rows=img.shape[0]
            cols=img.shape[1]
        
        img=pad_or_clip(img, rows, cols)
        labs=label(img, background=0)
        cleaned= remove_small_objects(labs, min_size=minsize)
        labs=label(cleaned, background=0)#re-label for continuous numbering
        stack.append(labs)
    return np.stack(stack, axis=0)


Function to rescale values of all visual features attached to btrack objects to interval 0-1

In [4]:
#Normalise/rescale all attached features to range 0-1
def normalise_visual_features(objects):
    import copy
    objects2=copy.copy(objects)#TODO doesn't seem to stop original objects changing
    #initialise variables to store min and max of each
    min_vals={}
    max_vals={}
    #Loop objects to find min and max for each property
    for obj in objects2:
        for prop in obj.properties:
            #collate minimum values
            if prop in min_vals:
                if min_vals[prop]>obj.properties[prop]:#if smaller than current min
                    min_vals[prop]=obj.properties[prop]
            else:
                min_vals[prop]=obj.properties[prop]#add dictionary entry with value
        
            #repeat for maximum values
            if prop in max_vals:
                if max_vals[prop]<obj.properties[prop]:
                    max_vals[prop]=obj.properties[prop] # if bigger than current max
            else:
                max_vals[prop]=obj.properties[prop]
    
    #Repeat the loop over objects and min-max scale the property values
    for obj in objects2:
        for prop in obj.properties:
            newprop=(obj.properties[prop]-min_vals[prop])/(max_vals[prop]-min_vals[prop])
            obj.properties[prop]=newprop
            
    return objects2

Set up paths to example data and ROI shapefile

In [5]:
PATH=os.path.join(Path().resolve(), "data")
#PATH

In [6]:
#Path to subsetting polygon shapefile
SubsetPolygon = os.path.join(PATH , 'Shapefile\Thwaites_Tracking_subset.shp')
#SubsetPolygon

In [7]:
#Example segmentations
files = glob.glob(os.path.join(PATH, 'Iceberg_Detections\Icebergs*.tif'))
#files

Open subset polygon for clipping of output rasters

In [8]:
with fiona.open(SubsetPolygon, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

#shapes

Build stack of segmented images including only objects greater than specified size (lower limit for detection algorithm is 63 pixels or 0.1km^2)

In [9]:
stack = segmentation_arr(files, minsize=2000)
stack.shape

(19, 5315, 3370)

Define visual features to calculate in call to regionprops and then those that we want to use in tracking updates

In [10]:
# features to be computed by regionprops
FEATURES_Regionprops = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "moments_hu"
]

# features to be used for tracking updates - note that each Hu moment we want to use needs to be specified individually - 
# hence can't just pass Features_Regionprops
FEATURES = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "moments_hu-0",
  "moments_hu-1",
  "moments_hu-2",
  "moments_hu-3",
  "moments_hu-4",
  "moments_hu-5",
  "moments_hu-6"
]

Generate btrack objects and rescale 0-1

In [11]:
obj_from_arr = btrack.utils.segmentation_to_objects(stack, properties=tuple(FEATURES_Regionprops), scale=(1., 1.))

[INFO][2022/06/13 03:32:02 PM] Localizing objects from segmentation...
[INFO][2022/06/13 03:32:08 PM] Objects are of type: <class 'dict'>
[INFO][2022/06/13 03:32:08 PM] ...Found 679 objects in 19 frames.


In [12]:
# inspect the first object
obj_from_arr[0]

,ID,x,y,z,t,dummy,states,label,area,major_axis_length,minor_axis_length,moments_hu-0,moments_hu-1,moments_hu-2,moments_hu-3,moments_hu-4,moments_hu-5,moments_hu-6
0,0,2076.339147,244.465088,0.0,0,False,7,5,113759,731.861407,245.973569,0.327515,0.068138,0.002445,0.000558,6.384127e-07,0.000118,1.346949e-07


In [13]:
#Normalise the visual features to range 0-1 # N.B. alters values for obj_from_arr also
obj_from_arr_norm=normalise_visual_features(obj_from_arr)

obj_from_arr_norm[0]

,ID,x,y,z,t,dummy,states,label,area,major_axis_length,minor_axis_length,moments_hu-0,moments_hu-1,moments_hu-2,moments_hu-3,moments_hu-4,moments_hu-5,moments_hu-6
0,0,2076.339147,244.465088,0.0,0,False,7,5,0.757991,0.920582,0.674873,0.062656,0.009749,0.000326,0.000081,0.001028,0.026465,0.981495






Initialise a tracker session using a context manager

In [14]:
with btrack.BayesianTracker() as tracker:

    # configure the tracker using a config file
    tracker.configure_from_file('data/ib_config.json')
    tracker.max_search_radius = 100

    # set up the features to use as a list (before appending)
    tracker.features = FEATURES

    # append the objects to be tracked
    tracker.append(obj_from_arr_norm)

    # set the volume
    tracker.volume=((0, 3370), (0, 5315), (-1e5, 1e5))
    #Z axis volume is set very large for 2D data)
    

    # track them (in interactive mode)
    tracker.track_interactive(step_size=50)

    # generate hypotheses and run the global optimizer
    #tracker.optimize()#docs suggest optimise motion model parameters first, then optimise hypothesis model

    tracker.export(os.path.join(PATH, 'tracking.h5'), obj_type='obj_type_1')

    # get the tracks in a format for napari visualization
    data, properties, graph = tracker.to_napari(ndim=3)
    
    tracks = tracker.tracks

[INFO][2022/06/13 03:34:27 PM] Loaded btrack: C:\Users\benevans\Anaconda3\envs\Py37_btrack\lib\site-packages\btrack\libs\libtracker.DLL
[INFO][2022/06/13 03:34:27 PM] btrack (v0.5.0) library imported
[INFO][2022/06/13 03:34:27 PM] Starting BayesianTracker session
[INFO][2022/06/13 03:34:27 PM] Loading configuration file: data/ib_config.json
[INFO][2022/06/13 03:34:27 PM] Objects are of type: <class 'list'>
[INFO][2022/06/13 03:34:27 PM] Starting tracking... 
[INFO][2022/06/13 03:34:27 PM] Update using: ['MOTION', 'VISUAL']
[INFO][2022/06/13 03:34:27 PM] Tracking objects in frames 0 to 19 (of 19)...
[INFO][2022/06/13 03:34:27 PM]  - Timing (Bayesian updates: 0.00ms, Linking: 0.00ms)
[INFO][2022/06/13 03:34:27 PM]  - Probabilities (Link: 0.99999, Lost: 1.00000)
[INFO][2022/06/13 03:34:27 PM] SUCCESS.
[INFO][2022/06/13 03:34:27 PM]  - Found 262 tracks in 19 frames (in 0.0s)
[INFO][2022/06/13 03:34:27 PM]  - Inserted 255 dummy objects to fill tracking gaps
[INFO][2022/06/13 03:34:27 PM] Op

Visualise tracks with Napari

In [15]:
viewer = napari.Viewer()
viewer.add_labels(stack, scale=(1., 1., 1.), name='Segmentation')
viewer.add_tracks(data, properties=properties, graph=graph, name='Tracks')

napari.manifest -> 'btrack:napari.yaml' could not be imported: Could not find file 'napari.yaml' in module 'btrack'


<Tracks layer 'Tracks' at 0x1e2909f0708>

Why does Napari generate many more segmentation timesteps than there are frames in the original stack of segmentations? 

Large objects appear not to have tracks assigned